In [32]:
import pandas as pd
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms

# 1) Transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3,
                           saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# 2) Dataset class (no change)
class UniversityStreetDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform):
        self.df = pd.read_csv(csv_file)
        self.img_dir = Path(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(self.img_dir/row["filename"]).convert("RGB")
        label = int(row["Region_ID"]) - 1
        return self.transform(img), label

# 3) Create datasets/loaders

# Train‐only data
train_ds = UniversityStreetDataset("labels_train.csv", "images_train", train_transforms)

# Val data *for training* (with augmentations)
val_ds_for_train = UniversityStreetDataset("labels_val.csv", "images_val", train_transforms)

# Val data *for evaluation* (deterministic)
val_ds_for_eval  = UniversityStreetDataset("labels_val.csv", "images_val", val_transforms)

# Combined train loader
# combined_ds = ConcatDataset([train_ds, val_ds_for_train])
# train_loader = DataLoader(combined_ds, batch_size=32, shuffle=True,
#                           num_workers=4, pin_memory=True)

train_loader = DataLoader(
                   train_ds,
                   batch_size=32,
                   shuffle=True,
                   num_workers=4,
                   pin_memory=True
               )

# Eval val loader (no shuffle!)
val_loader = DataLoader(val_ds_for_eval, batch_size=32, shuffle=False,
                        num_workers=4, pin_memory=True)


In [33]:
# Cell 1: full end-to-end EfficientNet-B0 with a two-hidden-layer head

import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# 1) Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) Load pretrained EfficientNet-B0
weights  = EfficientNet_B0_Weights.IMAGENET1K_V1
backbone = efficientnet_b0(weights=weights)

# 3) Replace the classifier with a deeper two-layer head
num_ftrs = backbone.classifier[1].in_features  # 1280

backbone.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_ftrs, 512),
    nn.ReLU(inplace=True),
    nn.Linear(512,     256),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(256,     15)    # 15 region classes
)

model = backbone.to(device)
print("Model ready, training all weights end-to-end for 100 epochs.")


Model ready, training all weights end-to-end for 100 epochs.


In [35]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

def train_one_epoch(model, loader, optimizer, device):
    model.train()
    running_loss = running_corrects = total = 0
    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        running_corrects += (logits.argmax(1) == labels).sum().item()
        total += imgs.size(0)
    return running_loss / total, running_corrects / total

def validate(model, loader, device):
    model.eval()
    running_loss = running_corrects = total = 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Val  ", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            loss = criterion(logits, labels)
            running_loss += loss.item() * imgs.size(0)
            running_corrects += (logits.argmax(1) == labels).sum().item()
            total += imgs.size(0)
    return running_loss / total, running_corrects / total

# ── Discriminative learning rates ─────────────────────────────────────────────
optimizer = Adam([
    {'params': model.classifier.parameters(), 'lr': 3e-5},   # head: base LR
    {'params': model.features.parameters(),   'lr': 3e-5},  # backbone: 10× smaller
], weight_decay=1e-4)

# Load your saved weights
model.load_state_dict(torch.load('best_model.pth', map_location=device))

# Now perturb (or re-init) the last two Linear layers:
noise_std = 1    # adjust this to control how “random” they get
with torch.no_grad():
    # layer at idx 3 → 512→256
    layer1 = model.classifier[3]
    # layer at idx 6 → 256→15
    layer2 = model.classifier[6]

    # Option A: add Gaussian noise around the existing weights
    layer1.weight.add_(torch.randn_like(layer1.weight) * noise_std)
    layer2.weight.add_(torch.randn_like(layer2.weight) * noise_std)
    if layer1.bias is not None:
        layer1.bias.add_(torch.randn_like(layer1.bias) * noise_std)
    if layer2.bias is not None:
        layer2.bias.add_(torch.randn_like(layer2.bias) * noise_std)

# Step LR: drop by 10× at epochs 10 and 20
scheduler = MultiStepLR(optimizer, milestones=[10, 20], gamma=0.1)

# ── Early stopping setup ──────────────────────────────────────────────────────
best_val_acc      = 0.0
patience          = 5
epochs_no_improve = 0
best_epoch        = 0

# ── Training loop with early stop & checkpoint ───────────────────────────────
for epoch in range(1, 31):
    # Train
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, device)
    # Validate
    vl_loss, vl_acc = validate(model, val_loader, device)
    # Step LR
    scheduler.step()

    print(f"Epoch {epoch:02d}  "
          f"Train: loss={tr_loss:.4f} acc={tr_acc:.4f}  "
          f" Val: loss={vl_loss:.4f} acc={vl_acc:.4f}")

    # Check for improvement
    if vl_acc > best_val_acc:
        best_val_acc      = vl_acc
        best_epoch        = epoch
        epochs_no_improve = 0
        # Save checkpoint
        torch.save(model.state_dict(), "best_model.pth")
        print(f"  ↳ New best val-acc ({vl_acc:.4f}), saving model.")
    else:
        epochs_no_improve += 1
        print(f"  ↳ No improvement for {epochs_no_improve} epoch(s).")

    # Early stopping
    if epochs_no_improve >= patience:
        print(f"Stopping early after epoch {epoch} (no improvement for {patience} consecutive epochs).")
        break

print(f"Training complete. Best val-acc: {best_val_acc:.4f} at epoch {best_epoch}.")


Epoch 01  Train: loss=143.6402 acc=0.2018   Val: loss=12.8205 acc=0.6883
  ↳ New best val-acc (0.6883), saving model.


Epoch 02  Train: loss=38.6320 acc=0.4625   Val: loss=3.2059 acc=0.8645
  ↳ New best val-acc (0.8645), saving model.


Epoch 03  Train: loss=19.2720 acc=0.5949   Val: loss=2.1320 acc=0.9187
  ↳ New best val-acc (0.9187), saving model.


Epoch 04  Train: loss=11.5000 acc=0.6943   Val: loss=1.6693 acc=0.9322
  ↳ New best val-acc (0.9322), saving model.


Epoch 05  Train: loss=8.6470 acc=0.7519   Val: loss=1.5221 acc=0.9431
  ↳ New best val-acc (0.9431), saving model.


...

Epoch 10  Train: loss=2.9458 acc=0.8843   Val: loss=0.7603 acc=0.9729
  ↳ No improvement for 1 epoch(s).


Epoch 11  Train: loss=2.6418 acc=0.8916   Val: loss=0.8216 acc=0.9729
  ↳ No improvement for 2 epoch(s).


Epoch 12  Train: loss=2.6670 acc=0.8921   Val: loss=0.8185 acc=0.9729
  ↳ No improvement for 3 epoch(s).


Epoch 13  Train: loss=2.6620 acc=0.8954   Val: loss=0.8255 acc=0.9729
  ↳ No improvement for 4 epoch(s).


Epoch 14  Train: loss=2.4909 acc=0.8990   Val: loss=0.8105 acc=0.9756
  ↳ No improvement for 5 epoch(s).
Stopping early after epoch 14 (no improvement for 5 consecutive epochs).
Training complete. Best val-acc: 0.9756 at epoch 9.


In [ ]:
import os
import csv
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T

# 0) device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) transforms for test/val (no flips/jitter)
test_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(256),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# 2) Test-only Dataset, catching all common image suffixes case-insensitively
class TestImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        # allowed suffixes:
        suffixes = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'}
        # list & filter
        all_files = os.listdir(root_dir)
        self.paths = sorted([
            os.path.join(root_dir, f)
            for f in all_files
            if os.path.splitext(f)[1].lower() in suffixes
        ])
        print(f"Found {len(self.paths)} images in {root_dir}")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(path)

# 3) Load model, set eval
model.eval()

# 4) predict region IDs on validation
pred_regions_val = []
with torch.no_grad():
    for imgs, _ in val_loader:
        imgs = imgs.to(device)
        logits = model(imgs)                 # [B,15]
        preds = logits.argmax(dim=1)         # [B]
        pred_regions_val.extend(preds.cpu().tolist())

n_val = len(pred_regions_val)
print(f"Validation: collected {n_val} region‐ID preds")


# 5) Build & load test set
test_ds = TestImageDataset("images_test", transform=test_transform)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=4)

pred_regions_test = []
with torch.no_grad():
    for imgs, fnames in test_loader:
        imgs = imgs.to(device)
        logits = model(imgs)
        preds = logits.argmax(dim=1)
        pred_regions_test.extend(preds.cpu().tolist())

print(f" -> Collected {len(pred_regions_test)} test predictions")

# 6) Write them out (assuming you already have n_val)
with open("region_predictions.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "Region_ID"])
    # write val preds [0..n_val-1]
    for i, r in enumerate(pred_regions_val):
        writer.writerow([i, r+1])
    # write test preds [n_val..]
    for offset, r in enumerate(pred_regions_test):
        writer.writerow([n_val + offset, r+1])


Found 369 images in images_test
 -> Collected 369 test predictions
